# 개발 노트
make_BuyHoldSell_point 고려할게 많다

# **Bitcoin Buy, Hold, Sell decision making project**
---

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Lambda
from keras import optimizers
from keras.utils import np_utils

import numpy as np
import math

Using TensorFlow backend.


**X : 시가,고가, 저가, 종가, 누적거래금액, 누적거래량**

In [0]:
rawData = np.loadtxt("/content/drive/My Drive/Colab Notebooks/priceData_1min.csv", delimiter=',')
#rawData = np.loadtxt("priceData.csv", delimiter=',')

In [0]:
X = rawData[:, 1:]
Y = np.zeros((len(X), 3))

### Normalization and Standardization

In [0]:
def normalize(data):
    return (data - np.min(data, axis = 0))/(np.max(data, axis = 0) - np.min(data, axis = 0))

In [0]:
def standardize(data) :
    return (data - np.mean(data, axis = 0))/np.std(data, axis = 0)

In [0]:
X = normalize(X)
X = standardize(X)

## Functions
---
make_BuyHoldSell_point()<br/>print_BuyHoldSell_point()

### make_BuyHoldSell_point
This function makes Buy, Hold, Sell point. (Y)
np.ndarray Y.shape is (len(X), 3)
- Buy : Y[ , 0] = 1
- Hold : Y[ , 1] = 1
- Sell : Y[ , 2] = 1

In [0]:
'''
Variable for make_BuyHoldSell_point
'''
candle_time = 1 # 1분봉
period = 60/1*12 #72 # (60분 / 10분봉) * 12시간
period = (int)(period/2)

In [0]:
make_point_length = 30 // candle_time # 최저, 최고점에서 몇분까지 Buy, Sell Point로 결정할지 정하는 변수
about_commision = 3 * 60 // candle_time # 수수료 발생을 고려하여, Buy, Sell Point가 3시간 이내일 경우 Buy, Sell Point를 Hold함
def make_BuyHoldSell_point(X, Y) :
    
    Y[:, :] = 0
        
    for i in range(period, len(X)-period) :

        arr = X[i-period:i+period, 3]

        minIndex = np.argmin(arr)
        maxIndex = np.argmax(arr)
        
        if abs(maxIndex - minIndex) > about_commision :
            for j in range(make_point_length) :
                if i-period + maxIndex + j < len(X) and i-period + minIndex + j < len(X) :
                    if Y[i-period + minIndex + j, 2] == 0 :
                        Y[i-period + minIndex + j, 0] = 1
                    if Y[i-period + maxIndex + j, 0] == 0 :
                        Y[i-period + maxIndex + j, 2] = 1
        
    for i in range(0, len(X)) :
        if Y[i, 0] == 0 and Y[i, 2] == 0 :
            Y[i, 1] = 1

### print_BuyHoldSell_point
This function shows time, BuyHoldSell, price.

In [0]:
def print_BuyHoldSell_point(X, Y) :
    for i in range(0, len(X)) :
        if Y[i, 0] == 1 :
            print(i, "\tBuy\t", X[i, 3], Y[i])
        elif Y[i, 2] == 1 :
            print(i, "\tSell\t", X[i, 3], Y[i])
        else :
            print(i, "\tHold\t", X[i, 3], Y[i])

## Building an RNN with LSTM
---

In [0]:
make_BuyHoldSell_point(X, Y)
#print_BuyHoldSell_point(X, Y)

In [0]:
print(len(X))

1010600


In [0]:
train_number = (int)(len(X) * 0.9)

X_train, X_test = X[:train_number], X[train_number:]
Y_train, Y_test = Y[:train_number], Y[train_number:]

#X_train = X_train.reshape((1, X_train.shape[0], X_train.shape[1]))
#X_test = X_test.reshape((1, X_test.shape[0], X_test.shape[1]))
#Y_train = Y_train.reshape((1, Y_train.shape[0], Y_train.shape[1]))
#Y_test = Y_test.reshape((1, Y_test.shape[0], Y_test.shape[1]))

In [0]:
print(X_train.shape)
print(Y_train.shape)

(909540, 6)
(909540, 3)


### Setting the RNN Parameters

In [0]:
batch_size = (int)(909540/1000)            # Batch size (you may try different values)
batch_length = (int)(train_number/1000)  # 9106
epochs = 15               # Epoch (you may try different values)
seq_len = 60 * 24 * 3     # 432 sequence data (Representing the last 3 days)
loss='mean_squared_error' # Since the metric is MSE/RMSE
opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01) #imizer = 'rmsprop'     # Recommended optimizer for RNN
activation = 'relu'       # Linear activation
input_shape=(None,6)      # Input dimension
output_dim = 32           # Output dimension

### Creating a Sequential Model

In [0]:
model = Sequential()
model.add(LSTM(units=output_dim, return_sequences=True, input_shape=input_shape))
model.add(Dense(units=output_dim,activation=activation))
model.add(LSTM(units=output_dim, return_sequences=False))
model.add(Dense(units=3,activation='softmax'))
model.compile(optimizer=opt,loss='categorical_crossentropy')

### Training the RNN

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 32)          4992      
_________________________________________________________________
dense_1 (Dense)              (None, None, 32)          1056      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 14,467
Trainable params: 14,467
Non-trainable params: 0
_________________________________________________________________


In [0]:
def trainning_Model(X_train, Y_train, batch_length, batch_size) :
    
    '''
    Make empty buy_list_number, hold_list_number, sell_list_number '''
    buy_list_number, hold_list_number, sell_list_number = [], [], []
    
    for i in range(seq_len, train_number) :
        if Y_train[i, 0] == 1 :
            buy_list_number.append(i)
        elif Y_train[i, 1] == 1 :
            hold_list_number.append(i)
        elif Y_train[i, 2] == 1 :
            sell_list_number.append(i)
    
    buy_list_number = np.array(buy_list_number)
    hold_list_number = np.array(hold_list_number)
    sell_list_number = np.array(sell_list_number)
    
    ''''''
    train_order_number = np.hstack((buy_list_number, hold_list_number, sell_list_number))
    np.random.shuffle(train_order_number)
    
    ''''''
    for epoch in range(epochs) :
    
        for batch in range(math.ceil(len(X_train)/batch_length)) :
          
            print("Epoch : %d / %d"%(epoch, epochs), "\tBatch : %d / %d"%(batch, math.ceil(len(X_train)/batch_length)), "\t")
          
            inputData = np.zeros((0, seq_len, 6))    # (m, seq_len, 6)
            outputData = np.zeros((0, 3))    # (m, seq_len, 6)
            
            for i in range(batch_length) :
              
                if i%100 == 0 :
                    print(i, end = ',')
                    
                if batch*batch_length + i < len(train_order_number) :
                    target_array_number = train_order_number[batch*batch_length + i]
                    X_input = X_train[target_array_number - seq_len : target_array_number, :]
                else :
                    X_input = X_train[target_array_number - seq_len :, :]

                X_input = X_input.reshape((1, X_input.shape[0], X_input.shape[1]))
                
                #print("X, Y, input, output :", X_input.shape, Y_train[target_array_number, :].shape, inputData.shape, outputData.shape)
                inputData = np.append(inputData, X_input, axis = 0)
                outputData = np.append(outputData, Y_train[target_array_number, :].reshape((1, 3)), axis = 0)

            model.fit(x=inputData, y=outputData, epochs=1, validation_split=0.00, batch_size = batch_size)
        
    model.save('predict_model.h5')

In [0]:
trainning_Model(X_train, Y_train, batch_length, batch_size)

Epoch : 0 / 15 	Batch : 0 / 1001 	
0,100,200,300,400,500,600,700,800,900,Epoch 1/1
909/909 [==============================] - 15s 16ms/step - loss: 0.8560
Epoch : 0 / 15 	Batch : 1 / 1001 	
0,100,200,300,400,500,600,700,800,900,Epoch 1/1
909/909 [==============================] - 14s 16ms/step - loss: 1.0045
Epoch : 0 / 15 	Batch : 2 / 1001 	
0,100,200,300,400,500,600,700,800,900,Epoch 1/1
909/909 [==============================] - 14s 16ms/step - loss: 0.9131
Epoch : 0 / 15 	Batch : 3 / 1001 	
0,100,200,300,400,500,600,700,800,900,Epoch 1/1
909/909 [==============================] - 14s 16ms/step - loss: 0.8546
Epoch : 0 / 15 	Batch : 4 / 1001 	
0,100,200,300,400,500,600,700,800,900,Epoch 1/1
909/909 [==============================] - 22s 24ms/step - loss: 0.8867
Epoch : 0 / 15 	Batch : 5 / 1001 	
0,100,200,300,400,500,600,700,800,900,Epoch 1/1
909/909 [==============================] - 15s 16ms/step - loss: 0.8799
Epoch : 0 / 15 	Batch : 6 / 1001 	
0,100,200,300,400,500,600,700,800,9

### Notebook

In [0]:
print_BuyHoldSell_point(X, Y)